In [1]:
using Distributed


In [2]:
using Gadfly


┌ Info: Precompiling Gadfly [c91e804a-d5a3-530f-b6f0-dfbca275c004]
└ @ Base loading.jl:1260
┌ Warning: Package FixedPointNumbers does not have Statistics in its dependencies:
│ - If you have FixedPointNumbers checked out for development and have
│   added Statistics as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with FixedPointNumbers
└ Loading Statistics into FixedPointNumbers from project dependency, future warnings for FixedPointNumbers are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Colors [5ae59095-9a9b-59fe-a467-6f913c188581]
│   exception = Required dependency FixedPointNumbers [53c48c17-4a7d-5ca2-90c5-79b7896eea93] failed to load from a cache file.
└ @ Base loading.jl:1041
┌ Warning: Package FixedPointNumbers does not have Statistics in its dependencies:
│ - If you have FixedPointNumbers checked out for development and ha

In [3]:
workervec = [ ("montecarlo@worker_1:22",1),
            ("montecarlo@worker_2:22",1),
            ("montecarlo@worker_3:22",1),
            ("montecarlo@worker_4:22",1)]

4-element Array{Tuple{String,Int64},1}:
 ("montecarlo@worker_1:22", 1)
 ("montecarlo@worker_2:22", 1)
 ("montecarlo@worker_3:22", 1)
 ("montecarlo@worker_4:22", 1)

In [4]:
addprocs(workervec; tunnel=true)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [5]:
println("Number procs: $(nprocs())")
println("Number of workers: $(nworkers())")

Number procs: 5
Number of workers: 4


In [ ]:
z = 0
x=0.5
y=0.5
# notamos que al darnos el bool el overload += lo cuenta como integer
println( (x*x + y*y) <=1 )
z += (x*x + y*y) <=1


In [23]:
@everywhere function pointsCircle(n,chart)
    # chart is a array of tuples
    n_in = 0
    for i=1:n
        x, y = rand(), rand()
        # this is a tuple (x,y)
#         if i<1000
#             push!(chart, (x,y)) # push the tuple to the array
#         end
        n_in += (x*x + y*y) <=1
    end
    return n_in
end

In [24]:
function pi_p(n,chart)
    # n quantity simulations distributed over p workers available
    p = nworkers()
    n_in = @distributed (+) for i=1:p # reduction call
        pointsCircle(n/p,chart)
    end
    return 4*n_in/n # Pi Aprox
end

pi_p (generic function with 2 methods)

In [ ]:
chart = Array{Tuple{Float64, Float64}}(undef, 0)

In [25]:
chart = Tuple{Float64, Float64}[]

0-element Array{Tuple{Float64,Float64},1}

In [26]:
chart = Tuple{Float64, Float64}[]
@time (4/10_000_000_000)*pointsCircle(10_000_000_000,chart)

114.070958 seconds (24.97 k allocations: 1.272 MiB)


3.141588092

In [ ]:
# X1 = (x->x[1]).(chart)
# Y1 = (y->y[2]).(chart)
# scatter(X1,Y1)

In [30]:
chart = Tuple{Float64, Float64}[]
@time pi_p(10_000_000_000,chart)

 70.427086 seconds (1.89 k allocations: 59.625 KiB)


3.1416122068

In [29]:
4/10_000_000_000

4.0e-10

For fun:
https://docs.julialang.org/en/v1/manual/metaprogramming/#man-macros-1

No existen magic commands in Ijulia pero tenemos la ayuda de macros, con estas y ayuda de magics.jl de Ijulia podemos hacer que salga babu frik diciendonos hurray.

Metaprogramming:

Macros
Macros provide a method to include generated code in the final body of a program. A macro maps a tuple of arguments to a returned expression, and the resulting expression is compiled directly rather than requiring a runtime eval call. Macro arguments may include expressions, literal values, and symbols.



In [ ]:
HTML("""
<head>
	<meta charset="utf-8">
	<title>Show Babu Frik </title>
	<link rel="stylesheet" href="toasty/toasty.css" />
</head>
<script src="https://code.jquery.com/jquery-1.11.2.min.js"></script>
<script src="toasty/jquery.toasty.js"></script>
""")

In [ ]:
macro javascript_str(s) display("text/javascript", s); end
macro hurray()
    return :(
        javascript"""
        $(document).ready( function(){
                  $("body").toasty();
                  $("body").toasty('pop');
        });
        """
    )
end

In [ ]:
@hurray